# Few-shot learning with SetFit and Argilla

This tutorial covers the case when there are no labels and we need to "bootstrap" a training dataset to use a few-shot approach - SetFit. We will also cover the use of Argilla for data labelling and quality assurance.

Here's a summary of the key steps in each approach:

**SetFit Framework:**

1. **Data Preparation**: The training data is read from a Parquet file and sampled to create a smaller dataset for demonstration purposes. This data is then converted into a Hugging Face `datasets` Dataset.

2. **Text Encoding**: The text data is encoded using a pre-trained sentence transformer model to obtain embeddings for each text instance. These will then be used for semantic search within Argilla...to speed up labeling.

After that we label the data manually...only a few examples.

3. **Training with SetFit**: The encoded text data is loaded into a SetFit dataset for text classification. The model, loss function, and trainer are instantiated, and the training process is initiated. After training, the model's performance is evaluated using the evaluation dataset.

**Traditional Machine Learning Approach:**

To compare performance we run a simple traditional classifier - using TFIDF and logistic regression.

1. **Data Loading**: The AG News dataset is loaded using the `load_dataset` function from Hugging Face datasets.

2. **Data Preprocessing**: The text data is split into training and test sets, and TF-IDF vectorization is applied to convert the text into numerical features. 

3. **Model Training and Evaluation**: A logistic regression model is trained on the TF-IDF transformed text data, and predictions are made on the test set. The performance of the model is evaluated using classification reports to assess its accuracy, precision, recall, and F1-score.



In [ ]:
%pip install argilla setfit -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [ ]:
import argilla as rg

In [ ]:
rg.init(
    api_url="https://YOUR-ARGILLA.hf.space", # add your space-url
    api_key="owner.apikey",
    workspace="admin"
)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.23.1 using a different client version (1.24.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [ ]:
from datasets import load_dataset, Dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer

In [ ]:
import pandas as pd

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
try:
    from argilla.utils.telemetry import tutorial_running
    tutorial_running()
except ImportError:
    print("Telemetry is introduced in Argilla 1.20.0 and not found in the current installation. Skipping telemetry.")

In [ ]:
## PREPPING THE DATA ##

# Read in the training data from a parquet file
data_train = pd.read_parquet('https://github.com/SDS-AAU/SDS-master/raw/master/M2/data/ag_news_unlabelled.pq')



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_train_sample = data_train.groupby('label').apply(lambda x: x.sample(n=30)).reset_index(drop=True)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convert the Pandas DataFrame to a Hugging Face `datasets` Dataset
dataset_news = Dataset.from_pandas(data_train_sample)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from sentence_transformers import SentenceTransformer

# Define fast version of sentence transformers
encoder = SentenceTransformer("intfloat/multilingual-e5-base", device="cuda")


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:

# Encode text field using batched computation
dataset_news = dataset_news.map(lambda batch: {"vectors": encoder.encode(batch["text"])}, batch_size=32, batched=True)

# Turn vectors into a dictionary
dataset_news = dataset_news.map(
    lambda r: {"vectors": {"mini-lm-sentence-transformers": r["vectors"]}}
)

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
unlabelled = rg.DatasetForTextClassification.from_datasets(dataset_news)

/usr/local/lib/python3.10/dist-packages/argilla/client/datasets.py:225: UserWarning: Following columns are not supported by the TextClassificationRecord model and are ignored: ['label']
  warnings.warn(


In [ ]:
rg.log(unlabelled, "news_sample_vecs")

Output()

120 records logged to ]8;id=764903;https://rjuro-unistra.hf.space/datasets/admin/news_sample_vecs\https://rjuro-unistra.hf.space/datasets/admin/news_sample_vecs]8;;\

BulkResponse(dataset='news_sample_vecs', processed=120, failed=0)

In [ ]:
# Load the handlabelled dataset from Argilla
train_ds = rg.load("news_sample_vecs").prepare_for_training()
test_ds = load_dataset("ag_news", split="test")

/usr/local/lib/python3.10/dist-packages/argilla/client/datasets.py:951: UserWarning: No label schema provided. Using all_labels: TextClassificationSettings(['Business', 'Sci/Tech', 'Sports', 'World']). We recommend providing a `TextClassificationSettings()` or setting `rg.configure_dataset_settings()`/`rg.load_dataset_settings()` to ensure reproducibility.
  warnings.warn(


In [ ]:
train_ds[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'id': '02de3f25-14f2-4b77-8c5f-c18129d8a550',
 'text': 'Stocks Rally on Lower Oil Prices Stocks rallied in quiet trading Wednesday as lower oil prices brought out buyers, countering a pair of government reports that gave a mixed picture of the economy.',
 'label': 0}

In [ ]:
train_ds.features

{'id': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['Business', 'Sci/Tech', 'Sports', 'World'], id=None)}

In [ ]:
test_ds.features

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}

In [ ]:
# Example label mappings for demonstration (replace these with your actual mappings)
label_mapping_a_to_b = {0: 2, 1: 3, 2:1, 3:0}

In [ ]:
# Function to apply label mapping
def apply_label_mapping(example, label_mapping):
    example['label'] = label_mapping[example['label']]
    return example

# Apply the mapping to align dataset_b labels with dataset_a
train_ds = train_ds.map(lambda x: apply_label_mapping(x, label_mapping_a_to_b))


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [ ]:
model = SetFitModel.from_pretrained("intfloat/multilingual-e5-base")

model = model.to('cuda')

# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,  # The number of text pairs to generate
)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
<ipython-input-59-b2bf454492d3>:6: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/28 [00:00<?, ? examples/s]

In [ ]:
# Train and evaluate
trainer.train()
metrics = trainer.evaluate()

***** Running training *****
  Num unique pairs = 1120
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 70


Step,Training Loss


***** Running evaluation *****


In [ ]:
metrics

{'accuracy': 0.8686842105263158}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

# Load the AG News dataset
dataset = load_dataset("ag_news")

# Prepare training and test sets
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']

test_texts = dataset['test']['text']
test_labels = dataset['test']['label']

# Create a TF-IDF vectorizer and logistic regression pipeline
model_lg = make_pipeline(
    TfidfVectorizer(stop_words='english'),
    LogisticRegression(max_iter=1000)
)

# Train the model
model_lg.fit(train_texts, train_labels)

# Predict on the test set
predicted_labels = model_lg.predict(test_texts)

# Evaluate the model
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1900
           1       0.96      0.98      0.97      1900
           2       0.89      0.88      0.88      1900
           3       0.89      0.90      0.89      1900

    accuracy                           0.92      7600
   macro avg       0.92      0.92      0.92      7600
weighted avg       0.92      0.92      0.92      7600

